# Submission File

# Install Libraries

In [ ]:
!rm -rf /kaggle/working/requirements
!cp -r -x ../input/gi-tract-models/packages/requirements /kaggle/working

!tar -czf /kaggle/working/requirements/efficientnet_pytorch-0.6.3.tar.gz -C /kaggle/working/requirements/efficientnet_pytorch-0.6.3/efficientnet_pytorch-0.6.3 .
!rm -rf /kaggle/working/requirements/efficientnet_pytorch-0.6.3

!tar -czf /kaggle/working/requirements/pretrainedmodels-0.7.4.tar.gz -C /kaggle/working/requirements/pretrainedmodels-0.7.4/pretrainedmodels-0.7.4 .
!rm -rf /kaggle/working/requirements/pretrainedmodels-0.7.4

In [ ]:
!pip install --no-index --find-links=/kaggle/working/requirements segmentation-models-pytorch
!pip install --no-index --find-links=/kaggle/working/requirements einops

In [ ]:
import sys
import cv2
import time
import torch
import torch.nn as nn
import logging
import numpy as np
import pandas as pd
import albumentations as A

from pathlib import Path
from tqdm.notebook import tqdm
from einops import repeat, rearrange
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [ ]:
tqdm.pandas()
logger = logging.getLogger(__name__)

# IO

In [ ]:
def load_image(image_path: Path) -> torch.Tensor:
    img = cv2.imread(str(image_path), cv2.IMREAD_UNCHANGED)
    img = repeat(img, "h w -> h w c", c=3)
    img = np.asarray(img, np.float32)
    img /= img.max()
    
    return img

In [ ]:
class SubmissionDataSet(Dataset):
    def __init__(self, image_paths, transforms=None):
        self.image_paths = image_paths
        self.transforms = transforms
        
    def __len__(self):
        return len(self.image_paths)
        
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        slice_number = img_path.stem[:10]
        case_day = img_path.parent.parent.name
        index = f"{case_day}_{slice_number}"
        
        image = load_image(img_path)
        height, width, chans = image.shape
        if self.transforms:
            data = self.transforms(image=image)
            image = data["image"]

        return image, index, height, width

In [ ]:
def get_metadata(row):
    data = row['id'].split('_')
    case = int(data[0].replace('case',''))
    day = int(data[1].replace('day',''))
    slice_ = int(data[-1])
    row['case'] = case
    row['day'] = day
    row['slice'] = slice_
    return row

def path2info(row):
    path = row['image_path']
    data = path.split('/')
    slice_ = int(data[-1].split('_')[1])
    case = int(data[-3].split('_')[0].replace('case',''))
    day = int(data[-3].split('_')[1].replace('day',''))
    width = int(data[-1].split('_')[2])
    height = int(data[-1].split('_')[3])
    row['height'] = height
    row['width'] = width
    row['case'] = case
    row['day'] = day
    row['slice'] = slice_
    return row

In [ ]:
sub_df = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/sample_submission.csv')
if not len(sub_df):
    debug = True
    sub_df = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/train.csv')[:1000*3]
    sub_df = sub_df.drop(columns=['class','segmentation']).drop_duplicates()   
else:
    debug = False
    sub_df = sub_df.drop(columns=['class','predicted']).drop_duplicates()
sub_df = sub_df.progress_apply(get_metadata, axis=1)

logger.info(f"Sub df length: {len(sub_df)}")
print(len(sub_df))
sub_df.head()

In [ ]:
from glob import glob 

if debug:
    paths = glob(f'/kaggle/input/uw-madison-gi-tract-image-segmentation/train/**/*png',recursive=True)
#     paths = sorted(paths)
else:
    paths = glob(f'/kaggle/input/uw-madison-gi-tract-image-segmentation/test/**/*png',recursive=True)
    
path_df = pd.DataFrame(paths, columns=['image_path'])
path_df = path_df.progress_apply(path2info, axis=1)

logger.info(f"Path df length: {len(path_df)}")
print(len(path_df))
path_df.head()

In [ ]:
test_df = sub_df.merge(path_df, on=['case','day','slice'], how='left')

logger.info(f"Test df length: {len(test_df)}")
print(len(test_df))
test_df.head()

In [ ]:
image_paths = [Path(x) for x in test_df["image_path"].tolist()]
assert len(image_paths) > 0, "Could not find any images"

INPUT_SIZE = (224, 224)
transforms = A.Compose([
    A.Resize(*INPUT_SIZE, cv2.INTER_NEAREST)
])

test_ds = SubmissionDataSet(image_paths, transforms)

In [ ]:
batch_size = 20
test_loader = DataLoader(
    test_ds,
    batch_size=batch_size,
    num_workers=4,
    pin_memory=True,
    drop_last=False
)

logger.info("Loaded dataset...")

In [ ]:
def running_length(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)



def convert_to_rle(mask: np.ndarray, id: str) -> pd.DataFrame:
    rows = []
    channel_index = {
        "large_bowel": 0,
        "small_bowel": 1,
        "stomach": 2,
    }

    for name, index in channel_index.items():
        submask = mask[..., index]
        # ordered from top to bottom, left to right
        flat_mask = submask.flatten()
        predicted = running_length(flat_mask)
        entry = {
            "id": id,
            "class": name,
            "predicted": predicted or None,
        }
        rows.append(entry)

    df = pd.DataFrame(rows)
    return df


def batch_rle_encoding(masks, indices, heights, widths):
    rles = []
    for mask, index, height, width in zip(masks, indices, heights, widths):
        mask_resized = cv2.resize(mask, (height, width), cv2.INTER_NEAREST)
        rle = convert_to_rle(mask_resized, index)
        rles.append(rle)
        
    return rles

# Inference

In [ ]:
import gc

thr = 0.5

frames = []

image_samples = []
mask_samples = []

device = "cuda" if torch.cuda.is_available() else "cpu"

@torch.no_grad()
def inference(data_loader, model_paths):
    for idx, (images, indices, heights, widths) in enumerate(tqdm(data_loader)):
        images = rearrange(images, "b h w c -> b c h w")
        images = images.to(device)
        
        cumulative = torch.zeros_like(images)
        
        for mp in model_paths:
            model = torch.load(mp, map_location=device)
            pred = model(images)
            probs = nn.Sigmoid()(pred)
            cumulative += probs
            
        probabilities = cumulative / len(model_paths)

        masks = (probabilities > thr).detach().cpu().numpy()
        masks = masks.astype(np.uint8)
        masks = rearrange(masks, "b c h w -> b h w c")

        if debug:
            image_samples.append(rearrange(images.detach().cpu().numpy(), "b c h w -> b h w c"))
            mask_samples.append(masks)


        heights = heights.detach().cpu().numpy()
        widths = widths.detach().cpu().numpy()

        rle = batch_rle_encoding(masks, indices, heights, widths)
        frames.extend(rle)
        logger.info(f"Completed step: {(idx + 1) * batch_size}")
        
        del images, masks
        gc.collect()
        torch.cuda.empty_cache()

In [ ]:
version = 2

model_dir = Path("/kaggle", "input", "gi-tract-models", str(version))
model_paths = list(model_dir.iterdir())
logger.info(f"Model Loaded {str(version)}")

In [ ]:
start = time.time()
inference(test_loader, model_paths)
end = time.time()

logger.info(f"Total time taken to perform inference: {end - start}s")

# Submission File

In [ ]:
sub = pd.concat(frames).reset_index(drop=True)
sub.columns = ["id", "class", "predicted"]
sub.to_csv("submission.csv", index=False)
logger.info("Saved submission...")
sub

In [ ]:
!rm -r /kaggle/working/requirements

In [ ]:
print(len(sub))
print(len(sub[~sub["predicted"].isna()]))
sub[~sub["predicted"].isna()]

## Prediction Visualisation

In [ ]:
import matplotlib.pyplot as plt

if len(image_samples) > 0:
    idx = np.random.randint(0, len(image_samples))

    image_batch = image_samples[idx]
    mask_batch = mask_samples[idx]

    for i in range(image_batch.shape[0]):
        fig, ax = plt.subplots(1, 2)
        ax[0].imshow(image_batch[i])
        ax[1].imshow(mask_batch[i] * 255)
        ax[0].axis('off')
        ax[1].axis('off')
        plt.tight_layout()
        plt.show()
